In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet_v2 import ResNet50V2

import warnings
warnings.filterwarnings(action='ignore') 

# 랜덤시드 고정시키기

np.random.seed(3)

In [3]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))

conv_base.trainable = False # 합성곱 기반 층 동결 - 훈련된 모델의 가중치가 업데이트 되지 않도록

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               10617088  
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 25,332,547
Trainable params: 10,617,859
Non-trainable params: 14,714,688
_________________________________________________________________


In [4]:
# 데이터 생성 - 훈련 데이터 각 2000개, 검증 데이터 각 300개, 테스트 데이터 각 300개

# 300*300 이미지 학습시키도록 한다. 
# 훈련 데이터 수가 클래스당 2000개이므로 배치 크기를 5로 지정하면 400번 배치를 수행해 한 에포크 돌아감

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/galaxy_train',target_size=(300, 300),batch_size=64,class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory('/content/drive/My Drive/galaxy_val',target_size=(300, 300),batch_size=64,class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/galaxy_test',target_size=(300, 300),batch_size=64, shuffle=False, class_mode='categorical')

Found 6000 images belonging to 3 classes.
Found 900 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


In [5]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit_generator(train_generator, epochs=40, validation_data=val_generator, verbose=1)

# 모델 평가하기 (훈련 데이터와 검증 데이터로 학습 후 맘에 들때까지 하이퍼 파리미터 (가중치 등) 조정하여 테스트 데이터로 평가)

print("-- Evaluate --")
scores = model.evaluate_generator(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 훈련의 정확도와 손실 그래프

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1, len(loss)+1)

plt.plot(epochs,acc,'r',label='Trainig accuracy')
plt.plot(epochs,val_acc,'b',label='Validation accuracy')
plt.ylim([0.5, 1.0])
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.ylim([0.0, 1.0])
plt.title('Training and validation loss')
plt.legend()

plt.show()

print("-- Predict --")
output = model.predict_generator(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)

f = open("result_ResNet50V2_3.txt", 'w')
np.savetxt('result_ResNet50V2_3.txt', output, fmt = '%0.3f')
f.close()

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
 6/94 [>.............................] - ETA: 27:00 - loss: 7.3208 - accuracy: 0.3177

KeyboardInterrupt: ignored

In [6]:
epochs = range(1, len(loss)+1)

plt.plot(epochs,acc,'r',label='Trainig accuracy')
plt.plot(epochs,val_acc,'b',label='Validation accuracy')
plt.ylim([0.5, 1.0])
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.ylim([0.0, 1.0])
plt.title('Training and validation loss')
plt.legend()

plt.show()

SyntaxError: ignored